In [ ]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
# Import RandomForestClassifier and GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# 1. Предобработка текста
def preprocess_text(text):
     if isinstance(text, str):
        # Удаляем ненужные символы и приводим текст к нижнему регистру
        text = text.lower()
        text = re.sub(r'\d+', '', text)  # Удаляем цифры
        text = re.sub(r'[^\w\s]', '', text)  # Удаляем знаки препинания
        return text
     else:
        return "" # Return empty string if not a string

# 2. Создание правил для каждого аспекта
def detect_aspects(review):
    aspects = {
        'Практика': 0,
        'Теория': 0,
        'Преподаватель': 0,
        'Технологии': 0,
        'Актуальность': 0
    }

    # Правила для "Практика"
    if re.search(r'\bпрактика|упражнения|задачи|задания|упражнения|навыки|опыт|стажировка|тренировка|применение|выполнение|работы|задания|практический|методика|реализация|кейсы|практическая|проект|лабораторные|процесс|результаты|тестирование\b', review):
        aspects['Практика'] = 1

    # Правила для "Теория"
    if re.search(r'\bтеория|информация|материал|концепция|методология|принципы|модели|исследования|основы|концептуальные|рамки|подходы|идеи|гипотезы|анализ|текст|лекции|курсы|философия|науки|аргументы|доказательства\b', review):
        aspects['Теория'] = 1

    # Правила для "Преподаватель"
    if re.search(r'\bпреподаватель|учитель|инструктор|лектор|преподаватель|учитель|лектор|инструктор|тренер|менеджер|куратор|наставник|педагог|эксперт|специалист|руководитель|обучающий|консультант|модератор|наставничество|образовательный|профессор|ассистент|сотрудник\b', review):
        aspects['Преподаватель'] = 1

    # Правила для "Технологии"
    if re.search(r'\bтехнологии|платформа|сайт|ресурсы|инструменты|технологии|инновации|инструменты|системы|платформы|программноеобеспечение|оборудование|устройства|автоматизация|алгоритмы|приложения|интерфейсы|разработка|техпроцесс|цифровизация|онлайнсервисы|машинноеобучение|искусственныйинтеллект|интернетвещей|базыданных\b', review):
        aspects['Технологии'] = 1

    # Правила для "Актуальность"
    if re.search(r'\bактуальность|обновление|современный|современность|релевантность|значимость|полезность|своевременность|ценность|приоритет|важность|популярность|новизна|тренды|спрос|перспективность|практичность|необходимость|обоснованность|соответствие|конкурентоспособность|злободневность\b', review):
        aspects['Актуальность'] = 1

    return aspects

# 3. Загрузка тренировочных и тестовых данных
train_df = pd.read_csv('/content/train_review2.csv')  # Загрузите ваши тренировочные данные
test_df = pd.read_csv('/content/test_review1.csv')    # Загрузите ваши тестовые данные

# Добавим колонку с предобработанными отзывами
train_df['Processed_Reviews'] = train_df['Reviews'].apply(preprocess_text)
test_df['Processed_Reviews'] = test_df['Reviews'].apply(preprocess_text)

# 4. Автоматическая разметка данных
train_aspects_data = train_df['Processed_Reviews'].apply(detect_aspects)
test_aspects_data = test_df['Processed_Reviews'].apply(detect_aspects)

# Преобразуем результаты в "DataFrame"
train_aspects_df = pd.DataFrame(train_aspects_data.tolist())
test_aspects_df = pd.DataFrame(test_aspects_data.tolist())

# Соединяем разметку с оригинальными данными
train_df = pd.concat([train_df, train_aspects_df], axis=1)
test_df = pd.concat([test_df, test_aspects_df], axis=1)

# 5. Извлечение текстов и меток для обучения и тестирования
X_train = train_df['Processed_Reviews']
y_train = train_df[['Практика', 'Теория', 'Преподаватель', 'Технологии', 'Актуальность']]

X_test = test_df['Processed_Reviews']
y_test = test_df[['Практика', 'Теория', 'Преподаватель', 'Технологии', 'Актуальность']]

# 6. Преобразование текстов в TF-IDF вектора
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)  # Обучаем векторизатор на тренировочных данных
X_test_tfidf = vectorizer.transform(X_test)  # Преобразуем тестовые данные

# 7. Обучение моделей Random Forest с гиперпараметрической настройкой
aspect_models = {}
aspects = ['Практика', 'Теория', 'Преподаватель', 'Технологии', 'Актуальность']

# Настройки для GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

for aspect in aspects:
    model = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, scoring='f1')
    grid_search.fit(X_train_tfidf, y_train[aspect])
    aspect_models[aspect] = grid_search.best_estimator_

# 8. Оценка моделей на тестировочных данных
for aspect in aspects:
    y_pred = aspect_models[aspect].predict(X_test_tfidf)
    print(f'Accuracy for {aspect}: {accuracy_score(y_test[aspect], y_pred)}')
    print(f'F1 Score for {aspect}: {f1_score(y_test[aspect], y_pred)}')

# 9. Предсказание аспектов на примере отзыва
def predict_aspects(review, vectorizer, aspect_models):
    review_processed = preprocess_text(review)
    review_tfidf = vectorizer.transform([review_processed])  # Преобразуем текст отзыва в TF-IDF

    predictions = {}
    for aspect, model in aspect_models.items():
        prediction = model.predict(review_tfidf)
        predictions[aspect] = prediction[0]

    return predictions

# Пример предсказания для отзыва
review = "Один из лучших курсов, хорошая теория, превосходная практика!"
prediction = predict_aspects(review, vectorizer, aspect_models)
print(prediction)